In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
full_df = pd.read_csv('../../data/full_df.csv')
train_df = pd.read_csv('../../data/train_df.csv')
prior_df = pd.read_csv('../../data/prior_df.csv')
# test_df = pd.read_csv('../../data/test_df.csv')

In [3]:
sample = pd.read_csv('../../data/sample_submission.csv')

In [4]:
sample

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259
...,...,...
74995,3420740,39276 29259
74996,3420877,39276 29259
74997,3420888,39276 29259
74998,3420989,39276 29259


In [9]:
# train_df.drop('Unnamed: 0',axis=1,inplace=True)
train_df.head() 

,order_id,user_id,product_id,reordered,target,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,product_ordered_vol,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
0,1187899,1,196,1,1,11,1,11,1.363636,37298,...,0.784615,11,1248.0,19.2,30.0,0.0,6.363636,1.000000,1.000000,0
1,1187899,1,10258,1,1,10,2,11,3.600000,2050,...,0.784615,11,1248.0,19.2,30.0,0.0,6.363636,0.909091,1.000000,0
2,1187899,1,13032,1,1,4,2,11,6.500000,3904,...,0.784615,11,1248.0,19.2,30.0,0.0,6.363636,0.363636,0.400000,0
3,1187899,1,25133,1,1,9,3,11,3.777778,6382,...,0.784615,11,1248.0,19.2,30.0,0.0,6.363636,0.818182,1.000000,0
4,1187899,1,26088,1,1,3,1,11,5.666667,2659,...,0.784615,11,1248.0,19.2,30.0,0.0,6.363636,0.272727,0.272727,0


In [5]:
full_df[full_df.duplicated(subset=['user_id','product_id'])]==True

# full_df[full_df.duplicated(subset=['user_id','product_id','order_id'])==True]

,Unnamed: 0,order_id,user_id,product_id,reordered,target,user_product_count,user_product_first_order,user_product_last_order,user_product_avg_basket_placement,...,user_reorder_ratio,user_total_orders,user_lifetime_days,user_avg_days_between_orders,user_max_time_between_orders,user_min_time_between_orders,user_avg_cart_size,user_product_order_rate,user_product_reorder_rate,user_product_last_time_product_ordered
1,True,False,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,True,False
2,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,True,False
3,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,True,False
4,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,True,False
5,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21470804,False,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
21470805,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
21470808,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21470809,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
X_train